# SGLang Native APIs

Apart from the OpenAI compatible APIs, the SGLang Runtime also provides its native server APIs. We introduce these following APIs:

- `/generate` (text generation model)
- `/get_model_info`
- `/get_server_info`
- `/health`
- `/health_generate`
- `/flush_cache`
- `/update_weights`
- `/encode`(embedding model)
- `/classify`(reward model)
- `/start_expert_distribution_record`
- `/stop_expert_distribution_record`
- `/dump_expert_distribution_record`

We mainly use `requests` to test these APIs in the following examples. You can also use `curl`.

## Launch A Server

In [1]:
import requests
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process


server_process, port = launch_server_cmd(
    "python3 -m sglang.launch_server --model-path qwen/qwen2.5-0.5b-instruct --host 0.0.0.0"
)

wait_for_server(f"http://localhost:{port}")

[2025-06-10 23:23:35] server_args=ServerArgs(model_path='qwen/qwen2.5-0.5b-instruct', tokenizer_path='qwen/qwen2.5-0.5b-instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='qwen/qwen2.5-0.5b-instruct', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, impl='auto', host='0.0.0.0', port=33978, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=590186916, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1,

[2025-06-10 23:23:48] Attention backend not set. Use fa3 backend by default.
[2025-06-10 23:23:48] Init torch distributed begin.


[2025-06-10 23:23:50] Init torch distributed ends. mem usage=0.00 GB


[2025-06-10 23:23:51] Load weight begin. avail mem=77.88 GB


[2025-06-10 23:23:51] Using model weights format ['*.safetensors']
[2025-06-10 23:23:52] No model.safetensors.index.json found in remote.
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


...

[2025-06-10 23:23:52] KV Cache is allocated. #tokens: 20480, K size: 0.12 GB, V size: 0.12 GB
[2025-06-10 23:23:52] Memory pool end. avail mem=69.48 GB


[2025-06-10 23:23:53] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=32768, available_gpu_mem=46.47 GB


[2025-06-10 23:23:53] INFO:     Started server process [3179152]
[2025-06-10 23:23:53] INFO:     Waiting for application startup.
[2025-06-10 23:23:53] INFO:     Application startup complete.
[2025-06-10 23:23:53] INFO:     Uvicorn running on http://0.0.0.0:33978 (Press CTRL+C to quit)
[2025-06-10 23:23:53] INFO:     127.0.0.1:50964 - "GET /v1/models HTTP/1.1" 200 OK


[2025-06-10 23:23:54] INFO:     127.0.0.1:50968 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-06-10 23:23:54] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-10 23:23:56] INFO:     127.0.0.1:50972 - "POST /generate HTTP/1.1" 200 OK
[2025-06-10 23:23:56] The server is fired up and ready to roll!


## Generate (text generation model)
Generate completions. This is similar to the `/v1/completions` in OpenAI API. Detailed parameters can be found in the [sampling parameters](./sampling_params.md).

In [2]:
url = f"http://localhost:{port}/generate"
data = {"text": "What is the capital of France?"}

response = requests.post(url, json=data)
print_highlight(response.json())

[2025-06-10 23:23:58] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-10 23:23:59] Decode batch. #running-req: 1, #token: 40, token usage: 0.00, cuda graph: False, gen throughput (token/s): 6.88, #queue-req: 0


[2025-06-10 23:23:59] Decode batch. #running-req: 1, #token: 80, token usage: 0.00, cuda graph: False, gen throughput (token/s): 121.75, #queue-req: 0


[2025-06-10 23:23:59] Decode batch. #running-req: 1, #token: 120, token usage: 0.01, cuda graph: False, gen throughput (token/s): 129.98, #queue-req: 0
[2025-06-10 23:23:59] INFO:     127.0.0.1:50976 - "POST /generate HTTP/1.1" 200 OK


## Get Model Info

Get the information of the model.

- `model_path`: The path/name of the model.
- `is_generation`: Whether the model is used as generation model or embedding model.
- `tokenizer_path`: The path/name of the tokenizer.

In [3]:
url = f"http://localhost:{port}/get_model_info"

response = requests.get(url)
response_json = response.json()
print_highlight(response_json)
assert response_json["model_path"] == "qwen/qwen2.5-0.5b-instruct"
assert response_json["is_generation"] is True
assert response_json["tokenizer_path"] == "qwen/qwen2.5-0.5b-instruct"
assert response_json.keys() == {"model_path", "is_generation", "tokenizer_path"}

[2025-06-10 23:23:59] INFO:     127.0.0.1:37974 - "GET /get_model_info HTTP/1.1" 200 OK


## Get Server Info
Gets the server information including CLI arguments, token limits, and memory pool sizes.
- Note: `get_server_info` merges the following deprecated endpoints:
  - `get_server_args`
  - `get_memory_pool_size` 
  - `get_max_total_num_tokens`

In [4]:
# get_server_info

url = f"http://localhost:{port}/get_server_info"

response = requests.get(url)
print_highlight(response.text)

[2025-06-10 23:23:59] INFO:     127.0.0.1:37980 - "GET /get_server_info HTTP/1.1" 200 OK


## Health Check
- `/health`: Check the health of the server.
- `/health_generate`: Check the health of the server by generating one token.

In [5]:
url = f"http://localhost:{port}/health_generate"

response = requests.get(url)
print_highlight(response.text)

[2025-06-10 23:23:59] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-10 23:24:00] INFO:     127.0.0.1:37988 - "GET /health_generate HTTP/1.1" 200 OK


In [6]:
url = f"http://localhost:{port}/health"

response = requests.get(url)
print_highlight(response.text)

[2025-06-10 23:24:01] INFO:     127.0.0.1:37996 - "GET /health HTTP/1.1" 200 OK


## Flush Cache

Flush the radix cache. It will be automatically triggered when the model weights are updated by the `/update_weights` API.

In [7]:
# flush cache

url = f"http://localhost:{port}/flush_cache"

response = requests.post(url)
print_highlight(response.text)

[2025-06-10 23:24:01] Cache flushed successfully!
[2025-06-10 23:24:01] INFO:     127.0.0.1:38000 - "POST /flush_cache HTTP/1.1" 200 OK


## Update Weights From Disk

Update model weights from disk without restarting the server. Only applicable for models with the same architecture and parameter size.

SGLang support `update_weights_from_disk` API for continuous evaluation during training (save checkpoint to disk and update weights from disk).


In [8]:
# successful update with same architecture and size

url = f"http://localhost:{port}/update_weights_from_disk"
data = {"model_path": "qwen/qwen2.5-0.5b-instruct"}

response = requests.post(url, json=data)
print_highlight(response.text)
assert response.json()["success"] is True
assert response.json()["message"] == "Succeeded to update model weights."

[2025-06-10 23:24:01] Start update_weights. Load format=auto
[2025-06-10 23:24:01] Update engine weights online from disk begin. avail mem=41.50 GB
[2025-06-10 23:24:01] Using model weights format ['*.safetensors']


[2025-06-10 23:24:01] No model.safetensors.index.json found in remote.
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  5.78it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  5.78it/s]



[2025-06-10 23:24:01] Update weights end.
[2025-06-10 23:24:01] Cache flushed successfully!
[2025-06-10 23:24:01] INFO:     127.0.0.1:38004 - "POST /update_weights_from_disk HTTP/1.1" 200 OK


In [9]:
# failed update with different parameter size or wrong name

url = f"http://localhost:{port}/update_weights_from_disk"
data = {"model_path": "qwen/qwen2.5-0.5b-instruct-wrong"}

response = requests.post(url, json=data)
response_json = response.json()
print_highlight(response_json)
assert response_json["success"] is False
assert response_json["message"] == (
    "Failed to get weights iterator: "
    "qwen/qwen2.5-0.5b-instruct-wrong"
    " (repository not found)."
)

[2025-06-10 23:24:01] Start update_weights. Load format=auto
[2025-06-10 23:24:01] Update engine weights online from disk begin. avail mem=41.50 GB


[2025-06-10 23:24:01] Failed to get weights iterator: qwen/qwen2.5-0.5b-instruct-wrong (repository not found).
[2025-06-10 23:24:01] INFO:     127.0.0.1:38008 - "POST /update_weights_from_disk HTTP/1.1" 400 Bad Request


In [10]:
terminate_process(server_process)

[2025-06-10 23:24:01] Child process unexpectedly failed with exitcode=9. pid=3179667


## Encode (embedding model)

Encode text into embeddings. Note that this API is only available for [embedding models](openai_api_embeddings.html#openai-apis-embedding) and will raise an error for generation models.
Therefore, we launch a new server to server an embedding model.

In [11]:
embedding_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model-path Alibaba-NLP/gte-Qwen2-1.5B-instruct \
    --host 0.0.0.0 --is-embedding
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-06-10 23:24:08] server_args=ServerArgs(model_path='Alibaba-NLP/gte-Qwen2-1.5B-instruct', tokenizer_path='Alibaba-NLP/gte-Qwen2-1.5B-instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Alibaba-NLP/gte-Qwen2-1.5B-instruct', chat_template=None, completion_template=None, is_embedding=True, enable_multimodal=None, revision=None, impl='auto', host='0.0.0.0', port=33312, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=371283995, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, bas

[2025-06-10 23:24:12] Downcasting torch.float32 to torch.float16.


[2025-06-10 23:24:19] Downcasting torch.float32 to torch.float16.


[2025-06-10 23:24:19] Overlap scheduler is disabled for embedding models.
[2025-06-10 23:24:19] Downcasting torch.float32 to torch.float16.
[2025-06-10 23:24:19] Attention backend not set. Use fa3 backend by default.
[2025-06-10 23:24:19] Init torch distributed begin.


[2025-06-10 23:24:20] Init torch distributed ends. mem usage=0.00 GB


...

[2025-06-10 23:24:25] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072, available_gpu_mem=57.64 GB


[2025-06-10 23:24:26] INFO:     Started server process [3182045]
[2025-06-10 23:24:26] INFO:     Waiting for application startup.
[2025-06-10 23:24:26] INFO:     Application startup complete.
[2025-06-10 23:24:26] INFO:     Uvicorn running on http://0.0.0.0:33312 (Press CTRL+C to quit)


[2025-06-10 23:24:26] INFO:     127.0.0.1:54552 - "GET /v1/models HTTP/1.1" 200 OK


[2025-06-10 23:24:27] INFO:     127.0.0.1:54568 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-06-10 23:24:27] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-10 23:24:27] INFO:     127.0.0.1:54580 - "POST /encode HTTP/1.1" 200 OK
[2025-06-10 23:24:27] The server is fired up and ready to roll!


In [12]:
# successful encode for embedding model

url = f"http://localhost:{port}/encode"
data = {"model": "Alibaba-NLP/gte-Qwen2-1.5B-instruct", "text": "Once upon a time"}

response = requests.post(url, json=data)
response_json = response.json()
print_highlight(f"Text embedding (first 10): {response_json['embedding'][:10]}")

[2025-06-10 23:24:31] Prefill batch. #new-seq: 1, #new-token: 4, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-06-10 23:24:31] INFO:     127.0.0.1:44054 - "POST /encode HTTP/1.1" 200 OK


In [13]:
terminate_process(embedding_process)

[2025-06-10 23:24:31] Child process unexpectedly failed with exitcode=9. pid=3182385
[2025-06-10 23:24:32] Child process unexpectedly failed with exitcode=9. pid=3182319


## Classify (reward model)

SGLang Runtime also supports reward models. Here we use a reward model to classify the quality of pairwise generations.

In [14]:
terminate_process(embedding_process)

# Note that SGLang now treats embedding models and reward models as the same type of models.
# This will be updated in the future.

reward_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model-path Skywork/Skywork-Reward-Llama-3.1-8B-v0.2 --host 0.0.0.0 --is-embedding
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-06-10 23:24:38] server_args=ServerArgs(model_path='Skywork/Skywork-Reward-Llama-3.1-8B-v0.2', tokenizer_path='Skywork/Skywork-Reward-Llama-3.1-8B-v0.2', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Skywork/Skywork-Reward-Llama-3.1-8B-v0.2', chat_template=None, completion_template=None, is_embedding=True, enable_multimodal=None, revision=None, impl='auto', host='0.0.0.0', port=33123, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=753408744, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, downloa

[2025-06-10 23:24:50] Overlap scheduler is disabled for embedding models.
[2025-06-10 23:24:50] Attention backend not set. Use flashinfer backend by default.
[2025-06-10 23:24:50] Init torch distributed begin.


[2025-06-10 23:24:50] Init torch distributed ends. mem usage=0.00 GB


[2025-06-10 23:24:51] Load weight begin. avail mem=61.47 GB


[2025-06-10 23:24:52] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


...

Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:01<00:01,  1.04it/s]


Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:02<00:00,  1.04it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.57it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.34it/s]

[2025-06-10 23:24:55] Load weight end. type=LlamaForSequenceClassification, dtype=torch.bfloat16, avail mem=47.43 GB, mem usage=14.04 GB.
[2025-06-10 23:24:55] KV Cache is allocated. #tokens: 20480, K size: 1.25 GB, V size: 1.25 GB
[2025-06-10 23:24:55] Memory pool end. avail mem=44.64 GB


[2025-06-10 23:24:56] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072, available_gpu_mem=44.07 GB


[2025-06-10 23:24:57] INFO:     Started server process [3184912]
[2025-06-10 23:24:57] INFO:     Waiting for application startup.
[2025-06-10 23:24:57] INFO:     Application startup complete.
[2025-06-10 23:24:57] INFO:     Uvicorn running on http://0.0.0.0:33123 (Press CTRL+C to quit)


[2025-06-10 23:24:58] INFO:     127.0.0.1:32812 - "GET /v1/models HTTP/1.1" 200 OK
[2025-06-10 23:24:58] INFO:     127.0.0.1:32816 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-06-10 23:24:58] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


In [15]:
from transformers import AutoTokenizer

PROMPT = (
    "What is the range of the numeric output of a sigmoid node in a neural network?"
)

RESPONSE1 = "The output of a sigmoid node is bounded between -1 and 1."
RESPONSE2 = "The output of a sigmoid node is bounded between 0 and 1."

CONVS = [
    [{"role": "user", "content": PROMPT}, {"role": "assistant", "content": RESPONSE1}],
    [{"role": "user", "content": PROMPT}, {"role": "assistant", "content": RESPONSE2}],
]

tokenizer = AutoTokenizer.from_pretrained("Skywork/Skywork-Reward-Llama-3.1-8B-v0.2")
prompts = tokenizer.apply_chat_template(CONVS, tokenize=False)

url = f"http://localhost:{port}/classify"
data = {"model": "Skywork/Skywork-Reward-Llama-3.1-8B-v0.2", "text": prompts}

responses = requests.post(url, json=data).json()
for response in responses:
    print_highlight(f"reward: {response['embedding'][0]}")

[2025-06-10 23:26:01] Prefill batch. #new-seq: 2, #new-token: 136, #cached-token: 2, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-06-10 23:26:01] INFO:     127.0.0.1:32826 - "POST /encode HTTP/1.1" 200 OK
[2025-06-10 23:26:01] The server is fired up and ready to roll!


[2025-06-10 23:26:15] INFO:     127.0.0.1:43602 - "POST /classify HTTP/1.1" 200 OK


In [16]:
terminate_process(reward_process)

[2025-06-10 23:26:15] Child process unexpectedly failed with exitcode=9. pid=3185286


## Capture expert selection distribution in MoE models

SGLang Runtime supports recording the number of times an expert is selected in a MoE model run for each expert in the model. This is useful when analyzing the throughput of the model and plan for optimization.

*Note: We only print out the first 10 lines of the csv below for better readability. Please adjust accordingly if you want to analyze the results more deeply.*

In [17]:
expert_record_server_process, port = launch_server_cmd(
    "python3 -m sglang.launch_server --model-path Qwen/Qwen1.5-MoE-A2.7B --host 0.0.0.0 --expert-distribution-recorder-mode stat"
)

wait_for_server(f"http://localhost:{port}")

[2025-06-10 23:26:21] server_args=ServerArgs(model_path='Qwen/Qwen1.5-MoE-A2.7B', tokenizer_path='Qwen/Qwen1.5-MoE-A2.7B', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Qwen/Qwen1.5-MoE-A2.7B', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, impl='auto', host='0.0.0.0', port=32425, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=250123680, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_level='

[2025-06-10 23:26:32] Attention backend not set. Use flashinfer backend by default.
[2025-06-10 23:26:32] Init torch distributed begin.


[2025-06-10 23:26:32] Init torch distributed ends. mem usage=0.00 GB


[2025-06-10 23:26:33] Load weight begin. avail mem=75.45 GB


[2025-06-10 23:26:33] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/8 [00:00<?, ?it/s]


...

Loading safetensors checkpoint shards: 100% Completed | 8/8 [00:06<00:00,  1.47it/s]
Loading safetensors checkpoint shards: 100% Completed | 8/8 [00:06<00:00,  1.25it/s]

[2025-06-10 23:26:40] Load weight end. type=Qwen2MoeForCausalLM, dtype=torch.bfloat16, avail mem=51.76 GB, mem usage=23.68 GB.
[2025-06-10 23:26:40] KV Cache is allocated. #tokens: 20480, K size: 1.88 GB, V size: 1.88 GB
[2025-06-10 23:26:40] Memory pool end. avail mem=47.85 GB


[2025-06-10 23:26:40] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=8192, available_gpu_mem=47.28 GB


[2025-06-10 23:26:41] INFO:     Started server process [3191918]
[2025-06-10 23:26:41] INFO:     Waiting for application startup.
[2025-06-10 23:26:41] INFO:     Application startup complete.
[2025-06-10 23:26:41] INFO:     Uvicorn running on http://0.0.0.0:32425 (Press CTRL+C to quit)
[2025-06-10 23:26:41] INFO:     127.0.0.1:57598 - "GET /v1/models HTTP/1.1" 200 OK


[2025-06-10 23:26:42] INFO:     127.0.0.1:57602 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-06-10 23:26:42] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-10 23:26:43] Using default MoE kernel config. Performance might be sub-optimal! Config file not found at /public_sglang_ci/runner-l1c-gpu-45/_work/sglang/sglang/python/sglang/srt/layers/moe/fused_moe_triton/configs/triton_3_3_1/E=60,N=1408,device_name=NVIDIA_H100_80GB_HBM3.json, you can create them with https://github.com/sgl-project/sglang/tree/main/benchmark/kernels/fused_moe_triton


In [18]:
response = requests.post(f"http://localhost:{port}/start_expert_distribution_record")
print_highlight(response)

url = f"http://localhost:{port}/generate"
data = {"text": "What is the capital of France?"}

response = requests.post(url, json=data)
print_highlight(response.json())

response = requests.post(f"http://localhost:{port}/stop_expert_distribution_record")
print_highlight(response)

response = requests.post(f"http://localhost:{port}/dump_expert_distribution_record")
print_highlight(response)

[2025-06-10 23:26:57] Resetting ExpertDistributionRecorder...
[2025-06-10 23:26:57] INFO:     127.0.0.1:57624 - "POST /start_expert_distribution_record HTTP/1.1" 200 OK


[2025-06-10 23:26:57] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 1, #queue-req: 0


[2025-06-10 23:26:58] INFO:     127.0.0.1:56874 - "POST /generate HTTP/1.1" 200 OK


[2025-06-10 23:26:58] INFO:     127.0.0.1:57618 - "POST /generate HTTP/1.1" 200 OK
[2025-06-10 23:26:58] INFO:     127.0.0.1:56880 - "POST /stop_expert_distribution_record HTTP/1.1" 200 OK
[2025-06-10 23:26:58] The server is fired up and ready to roll!


[2025-06-10 23:26:59] Write expert distribution to /tmp/expert_distribution_recorder_1749598019.477921.pt
[2025-06-10 23:26:59] Resetting ExpertDistributionRecorder...
[2025-06-10 23:26:59] INFO:     127.0.0.1:56886 - "POST /dump_expert_distribution_record HTTP/1.1" 200 OK


In [19]:
terminate_process(expert_record_server_process)

[2025-06-10 23:26:59] Child process unexpectedly failed with exitcode=9. pid=3192220
